In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lib.generate_timeseries import Periodic_1d
import torch
from collections import Counter
from matplotlib import cm
from torch.distributions import uniform
import pandas as pd
import seaborn as sns
import sys
import json
import glob
import os
os.chdir("/Users/luisa/Desktop/nygc/cluster/projects/fl4tsf/bin")
sys.path.append(os.getcwd()) 
sns.set(style='whitegrid')
from lib.dataset_utils import *
%matplotlib inline
%load_ext autoreload
%autoreload 2
from plot_utils import read_loss_file
from lib.physionet import PhysioNet, variable_time_collate_fn, get_data_min_max
from sklearn import model_selection
from torch.utils.data import DataLoader
import lib.utils as utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
path_prefix = "/Users/luisa/Desktop/nygc/cluster/projects/fl4tsf/data"
prefix = "physionet"
batch_size = 64
classif = False

In [25]:
#################################################################
# Physionet Dataset
#################################################################
quantization = 0.1 # quantization of time stamps in 0.1 hours
n = 10000 # number of samples to use from the dataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_dataset_obj = PhysioNet('../data/physionet', train=True, 
                                quantization = quantization,
                                download=True, n_samples = min(10000, n), 
                                device = device)
# Use custom collate_fn to combine samples with arbitrary time observations.
# Returns the dataset along with mask and time steps
test_dataset_obj = PhysioNet('../data/physionet', train=False, 
                                quantization = quantization,
                                download=True, n_samples = min(10000,n), 
                                device = device)

# Combine and shuffle samples from physionet Train and physionet Test
total_dataset = train_dataset_obj[:len(train_dataset_obj)]


Using downloaded and verified file: ../data/physionet/PhysioNet/raw/Outcomes-a.txt
Using downloaded and verified file: ../data/physionet/PhysioNet/raw/set-a.tar.gz?download


EOFError: Compressed file ended before the end-of-stream marker was reached

In [23]:

if not classif:
    # Concatenate samples from original Train and Test sets
    # Only 'training' physionet samples are have labels. Therefore, if we do classifiction task, we don't need physionet 'test' samples.
    total_dataset = total_dataset + test_dataset_obj[:len(test_dataset_obj)]

# Shuffle and split
train_data, test_data = model_selection.train_test_split(total_dataset, train_size= 0.8, 
    random_state = 42, shuffle = True)

record_id, tt, vals, mask, labels = train_data[0]

n_samples = len(total_dataset)
input_dim = vals.size(-1)

batch_size = min(min(len(train_dataset_obj),batch_size), n)
data_min, data_max = get_data_min_max(total_dataset)

train_dataloader = DataLoader(train_data, batch_size= batch_size, shuffle=False, 
    collate_fn= lambda batch: variable_time_collate_fn(batch, args, device, data_type = "train",
        data_min = data_min, data_max = data_max))
test_dataloader = DataLoader(test_data, batch_size = n_samples, shuffle=False, 
    collate_fn= lambda batch: variable_time_collate_fn(batch, args, device, data_type = "test",
        data_min = data_min, data_max = data_max))

attr_names = train_dataset_obj.params
data_objects = {"dataset_obj": train_dataset_obj, 
            "train_dataloader": utils.inf_generator(train_dataloader), 
            "test_dataloader": utils.inf_generator(test_dataloader),
            "input_dim": input_dim,
            "n_train_batches": len(train_dataloader),
            "n_test_batches": len(test_dataloader),
            "attr": attr_names, #optional
            "classif_per_tp": False, #optional
            "n_labels": 1} #optional